In [1]:
#Import.
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
from sklearn.model_selection import train_test_split
from itertools import combinations, product
from numba import njit, vectorize
import numba as nb
from ast import literal_eval
import random

In [2]:
#Funciones.
def load_ib2():
    name = 'ib'
    V = pd.read_csv(name+'.csv', sep=';')
    V = pd.merge(V, 
                 ITEM_DATA.loc[:,['item_id', 'domain_code', 'mexico']], 
                 how='left', 
                 left_on='ib', 
                 right_on='item_id').drop(columns=['item_id'])
    return np.array(V['ib']), np.array(V['domain_code']), np.array(V['mexico'])

def load_chosen_predictors(folder, predictors=[], nan_treat=False, k=10):
    PRED = {}
    pred = {}
    PRED1 = {}
    pred1 = {}
    PRED2 = {}
    pred2 = {}
    for fname in tqdm(os.listdir(folder)):
        name = fname.replace('_r413162','').replace('.csv','')
        if name in predictors or len(predictors) == 0:
            if '_r413162' in fname:
                PRED[name] = pd.read_csv(folder+'/'+fname)
                pred[name] = np.array(PRED[name])
                pred[name] = pred[name][:,1:]
            else:
                PRED[name] = pd.read_csv(folder+'/'+fname, sep=';')
                pred[name] = np.array(PRED[name])
            if nan_treat:
                pred[name][np.isnan(pred[name])] = -1
            for j, p in enumerate(['pred'+str(i) for i in range(k)]):
                if j == 0:
                    PRED1[name] = pd.merge(PRED[name][p], 
                        ITEM_DATA.loc[:,['item_id', 'domain_code']],
                        how='left',
                        left_on=p,
                        right_on='item_id').drop(columns=[p, 'item_id'])
                    PRED2[name] = pd.merge(PRED[name][p], 
                        ITEM_DATA.loc[:,['item_id', 'mexico']],
                        how='left',
                        left_on=p,
                        right_on='item_id').drop(columns=[p, 'item_id'])
                else:
                    TEMP = pd.merge(PRED[name][p], 
                        ITEM_DATA.loc[:,['item_id', 'domain_code']],
                        how='left',
                        left_on=p,
                        right_on='item_id').drop(columns=[p, 'item_id'])
                    TEMP1 = pd.merge(PRED[name][p], 
                        ITEM_DATA.loc[:,['item_id', 'mexico']],
                        how='left',
                        left_on=p,
                        right_on='item_id').drop(columns=[p, 'item_id'])
                    PRED1[name] = pd.concat([PRED1[name], TEMP], axis=1)
                    PRED2[name] = pd.concat([PRED2[name], TEMP1], axis=1)
                PRED1[name].loc[PRED1[name]['domain_code'].isnull(), 'domain_code'] = -1
                PRED1[name] = PRED1[name].rename(columns={'domain_code': p})
                #PRED2[name].loc[PRED2[name]['mexico'].isnull(), 'mexico'] = False
                PRED2[name] = PRED2[name].rename(columns={'mexico': p})
            PRED1[name] = PRED1[name].astype('int64')
            pred1[name] = np.array(PRED1[name])
            #PRED2[name] = PRED2[name].astype('bool')
            pred2[name] = np.array(PRED2[name])
    return PRED, pred, PRED1, pred1, PRED2, pred2

def load_chosen_predictors_pred(folder, predictors=[], nan_treat=False, k=10):
    PRED = {}
    pred = {}
    for fname in tqdm(os.listdir(folder)):
        name = fname.replace('_pred','').replace('.csv','')
        if name in predictors or len(predictors) == 0:
            PRED[name] = pd.read_csv(folder+'/'+fname, sep=';')
            pred[name] = np.array(PRED[name])
            if nan_treat:
                pred[name][np.isnan(pred[name])] = -1
    return PRED, pred

def get_doms_and_mket(Y_TEST, k=10):
    for j, p in enumerate(['pred'+str(i) for i in range(k)]):
        if j == 0:
            Y_TEST_DO = pd.merge(Y_TEST[p], 
                ITEM_DATA.loc[:,['item_id', 'domain_code']],
                how='left',
                left_on=p,
                right_on='item_id').drop(columns=[p, 'item_id'])
            Y_TEST_MA = pd.merge(Y_TEST[p], 
                ITEM_DATA.loc[:,['item_id', 'mexico']],
                how='left',
                left_on=p,
                right_on='item_id').drop(columns=[p, 'item_id'])
        else:
            TEMP = pd.merge(Y_TEST[p], 
                ITEM_DATA.loc[:,['item_id', 'domain_code']],
                how='left',
                left_on=p,
                right_on='item_id').drop(columns=[p, 'item_id'])
            TEMP1 = pd.merge(Y_TEST[p], 
                ITEM_DATA.loc[:,['item_id', 'mexico']],
                how='left',
                left_on=p,
                right_on='item_id').drop(columns=[p, 'item_id'])
            Y_TEST_DO = pd.concat([Y_TEST_DO, TEMP], axis=1)
            Y_TEST_MA = pd.concat([Y_TEST_MA, TEMP1], axis=1)
        Y_TEST_DO.loc[Y_TEST_DO['domain_code'].isnull(), 'domain_code'] = -1
        Y_TEST_DO = Y_TEST_DO.rename(columns={'domain_code': p})
        Y_TEST_MA = Y_TEST_MA.rename(columns={'mexico': p})
    Y_TEST_DO = Y_TEST_DO.astype('int64')
    y_test_do = np.array(Y_TEST_DO)
    y_test_ma = np.array(Y_TEST_MA)
    return Y_TEST_DO, y_test_do, Y_TEST_MA, y_test_ma

def get_doms(ii):
    II = pd.DataFrame(ii, columns=['items'])
    II_DO = pd.merge(II['items'], 
        ITEM_DATA.loc[:,['item_id', 'domain_code']],
        how='left',
        left_on='items',
        right_on='item_id').drop(columns=['items', 'item_id'])
    II_DO.loc[II_DO['domain_code'].isnull(), 'domain_code'] = -1
    II_DO = II_DO.rename(columns={'domain_code': 'doms'})
    return np.array(II_DO).reshape(-1)

def eval_feature(a):
    e = 0
    for i in range(a.size):
        e += a[i] / np.log(i + 2)
    return e

@njit(parallel=True)
def get_y_hat(ii, rr, k=10):
    target = np.unique(ii)
    size = target.size
    temp = np.zeros(size)
    for i in range(size):
        #temp[i] = rr[ii == target[i]].max()
        temp[i] = rr[ii == target[i]].sum()
    return target[temp.argsort()][-k:]

@njit(parallel=True)
def compare(a, b):
    return a == b

@vectorize(nopython=True, target='parallel')
def less_than_13(a):
    a[a > 12] == 12
    return a

def gen_sales(y_hat, cdom, cmket):
    if cdom == -1:
        cdom = 2082
    if cmket == 1:
        target = brs[brs_do == cdom]
    else:
        target = mex[mex_do == cdom]
    return target[np.logical_not(np.isin(target, y_hat, assume_unique=True))]

In [3]:
ITEM_DATA = pd.read_csv('item_data.csv', sep=';')
ITEM_DATA.loc[ITEM_DATA['product_id'] == 0, 'product_id'] = -1
ITEM_DATA['domain_code'], domain_uniques = pd.factorize(ITEM_DATA['domain_id'], sort=True)
ITEM_DATA['category_code'], category_uniques = pd.factorize(ITEM_DATA['category_id'], sort=True)

In [4]:
ib, ib_do, ib_ma = load_ib2()

In [5]:
#chosen_predictors = ['dim_bc',
#                     'dim_liv',
#                     'dim_miv',
#                     'dim_odom_13',
#                     'dim_opro_13',
#                     'dim_ocat_13',
#                     'dim_odom_16',
#                     'dim_opro_16',
#                     'dim_ocat_16',
#                     'dim_sdom_31',
#                     'dim_spro_31',
#                     'dim_scat_31',
#                     'dim_sdom_13',
#                     'dim_spro_13',
#                     'dim_scat_13',
#                     'dim_tbd',
#                     'dim_tbd2',
#                     'dim_tbc',
#                     'dim_tbc2',
#                     'dim_tbp',
#                     'dim_tbp2',
#                     'dim_vc',
#                     'dim_sss']

chosen_predictors = ['dim_miv', 'dim_tbc', 'dim_liv', 'dim_ocat_16', 'dim_bc', 'dim_sss', 'dim_tbp', 'dim_vc']

PRED, pred, PRED1, pred1, PRED2, pred2 = load_chosen_predictors('C:/data/meli_challenge/predictors', chosen_predictors)

In [6]:
#Split.
main_index = [x for x in range(len(ib))]
idx_train, idx_test = train_test_split(main_index, test_size=0.5, random_state=45)
idx_train = sorted(idx_train)
idx_test = sorted(idx_test)
ib_train = ib[idx_train]
ib_test = ib[idx_test]
ib_do_train = ib_do[idx_train]
ib_do_test = ib_do[idx_test]
ib_ma_train = ib_ma[idx_train]
ib_ma_test = ib_ma[idx_test]
pred_train = {}
pred1_train = {}
pred2_train = {}
pred_test = {}
for f in chosen_predictors:
    pred_train[f] = pred[f][idx_train]
    pred1_train[f] = pred1[f][idx_train]
    pred2_train[f] = pred2[f][idx_train]
    pred_test[f] = pred[f][idx_test]

In [7]:
#Mercado y dominio visitado.
DFF = pd.read_csv('dff.csv', sep=';')
mket_train = np.array(DFF['mercado'])[idx_train]
mket_test = np.array(DFF['mercado'])[idx_test]
DMV = pd.read_csv('dmv.csv', sep=';')
dom_train = np.array(DMV['vdomain']).reshape(-1)[idx_train]
dom_test = np.array(DMV['vdomain']).reshape(-1)[idx_test]

In [8]:
#Tops.
top_mex = [716822, 429798, 1303554, 806452, 1767506, 1276232, 1796243, 795217, 1930193, 1139737]
top_brs = [1587422, 1803710, 10243, 548905, 1906937, 1361154, 1716388, 725371, 859574, 290755]
MEX = pd.read_csv('mex.csv', sep=';')
BRS = pd.read_csv('brs.csv', sep=';')
mex = np.array(MEX['item'])
brs = np.array(BRS['item'])
mex_do = np.array(MEX['domain'])
brs_do = np.array(BRS['domain'])

In [9]:
#Clusters.
clusters = pd.read_csv('clusters_maradona_v2.csv', sep=';')
train_test = pd.read_csv('meli_segment_v2.csv', sep=';')
clusters = pd.concat([clusters['cs_a3'], train_test], axis=1)
clusters = clusters.loc[clusters['segment'] == 'train']
cluster = np.array(clusters['cs_a3']).astype('int32')
cluster_train = cluster[idx_train]
cluster_test = cluster[idx_test]
cl_train = np.unique(cluster_train, return_counts=True)
cl_test = np.unique(cluster_test, return_counts=True)

In [ ]:
#Trampa cluster.
#clusters = pd.read_csv('dfs.csv', sep=';')
#clusters['segmento'] = 0
#clusters.loc[clusters['v_len'] != 0, 'segmento'] = 1
#cluster = np.array(clusters['segmento']).astype('int32')
#cluster_train1 = cluster[idx_train]
#cluster_test1 = cluster[idx_test]
#cl_train1 = np.unique(cluster_train1, return_counts=True)
#cl_test1 = np.unique(cluster_test1, return_counts=True)

In [10]:
#Grilla.
ibr = ib_train.reshape(ib_train.size,1).repeat(10, axis=1)
ibr_do = ib_do_train.reshape(ib_do_train.size,1).repeat(10, axis=1)
ibr_ma = ib_ma_train.reshape(ib_ma_train.size,1).repeat(10, axis=1)

grilla = {}
for k in tqdm(range(len(cl_train[0]))):
    eval = {}
    eval1 = {}
    eval2 = {}
    performance = {}
    feat_perf = {}
    for i in pred:
            eval[i] = ibr[cluster_train == cl_train[0][k]] == pred_train[i][cluster_train == cl_train[0][k]]
            eval1[i] = ibr_do[cluster_train == cl_train[0][k]] == pred1_train[i][cluster_train == cl_train[0][k]]
            eval2[i] = ibr_ma[cluster_train == cl_train[0][k]] == pred2_train[i][cluster_train == cl_train[0][k]]
            performance[i] = eval[i].mean(axis=0) + (eval1[i] * eval2[i]).mean(axis=0) / 12
            feat_perf.update({i: performance[i]})
    grilla.update({cl_train[0][k]: feat_perf})

In [11]:
#Performance.
eval = {}
eval1 = {}
eval2 = {}
performance = {}
feat_perf = {}
ibr = ib_train.reshape(ib_train.size,1).repeat(10, axis=1)
ibr_do = ib_do_train.reshape(ib_do_train.size,1).repeat(10, axis=1)
ibr_ma = ib_ma_train.reshape(ib_ma_train.size,1).repeat(10, axis=1)

for i in pred_train:
        eval[i] = ibr == pred_train[i]
        eval1[i] = ibr_do == pred1_train[i]
        eval2[i] = ibr_ma == pred2_train[i]
        performance[i] = eval[i].mean(axis=0) + (eval1[i] * eval2[i]).mean(axis=0) / 12
        feat_perf.update({i: performance[i]})

In [12]:
#Versiones. VER BIEN AL EMPEZAR DE NUEVO.
grupoA = list(chosen_predictors)
grupoB = []
c_versiones = [[x] for x in grupoA]

In [12]:
#Versiones.
#grupoA = ['dim_bc',
#         'dim_odom_13',
#         'dim_opro_13',
#         'dim_ocat_13',
#         'dim_odom_16',
#         'dim_opro_16',
#         'dim_ocat_16',
#         'dim_sdom_31',
#         'dim_spro_31',
#         'dim_scat_31',
#         'dim_sdom_13',
#         'dim_spro_13',
#         'dim_scat_13',
#         'dim_tbd',
#         'dim_tbd2',
#         'dim_tbc2',
#         'dim_tbp',
#         'dim_tbp2',
#         'dim_vc',
#         'dim_sss']

grupoA = ['dim_bc']

#grupoB = ['dim_miv', 'dim_tbc', 'dim_liv']

grupoB = ['dim_miv', 'dim_tbc', 'dim_liv', 'dim_sss', 'dim_vc', 'dim_ocat_16', 'dim_tbp']

c_versiones = []
for i in grupoA:
    c_versiones.append(grupoB + [i]) 
    
c_versiones

[['dim_miv',
  'dim_tbc',
  'dim_liv',
  'dim_sss',
  'dim_vc',
  'dim_ocat_16',
  'dim_tbp',
  'dim_bc']]

In [13]:
#Predicción.
versiones = []
k = 10
nDGC_version = []
max_nDCG = 0
last_nDCG = -1

while len(grupoA) > 0 and max_nDCG > last_nDCG:
    
    last_nDCG = max_nDCG
    max_nDCG = None
    for v in tqdm(c_versiones):
        c_feature = v[-1]
        
        y_test = []
        for i in tqdm(range(ib_test.size)):
            ccluster = cluster_test[i]
            cdom = dom_test[i]
            cmket = mket_test[i]
            for j, f in enumerate(v):
                if ccluster in grilla:
                    cref = grilla[ccluster][f]
                else:
                    alt_cluster = [x for x in grilla if abs(x - ccluster) == min([abs(x - ccluster) for x in grilla])][0]
                    cref = grilla[alt_cluster][f]
                citems = pred_test[f][i]
                mask = (cref > 0) & (citems >= 0)
                cref = cref[mask]
                citems = citems[mask]
                if j == 0:
                    ranks = cref
                    items = citems
                else:
                    ranks = np.concatenate([ranks, cref])
                    items = np.concatenate([items, citems])
            y_hat = list(get_y_hat(items, ranks)[::-1])
            if len(y_hat) < k:
                size = k - len(y_hat)
                ext = list(gen_sales(y_hat, cdom, cmket)[:size])
                y_hat.extend(ext)
                if len(y_hat) < k:
                    size = k - len(y_hat)
                    if cmket == 1:
                        ext = [x for x in top_mex if x not in y_hat][:size]
                    else:
                        ext = [x for x in top_brs if x not in y_hat][:size]
                    y_hat.extend(ext)
            y_test.append(y_hat)

        Y_TEST = pd.DataFrame(y_test, columns=['pred'+str(i) for i in range(k)])
        Y_TEST_DO, y_test_do, Y_TEST_MA, y_test_ma = get_doms_and_mket(Y_TEST)
        cond1 = compare(np.array(y_test), ib_test.reshape(ib_test.size,1).repeat(10, axis=1))
        cond2 = compare(y_test_do, ib_do_test.reshape(ib_do_test.size,1).repeat(10, axis=1))
        cond3 = y_test_ma == ib_ma_test.reshape(ib_ma_test.size,1).repeat(10, axis=1)
        numerador = cond1 * 12 + cond2 * cond3
        numerador[numerador > 12] = 12

        lns = {0: np.log(2),
                1: np.log(3),
                2: np.log(4),
                3: np.log(5),
                4: np.log(6),
                5: np.log(7),
                6: np.log(8),
                7: np.log(9),
                8: np.log(10),
                9: np.log(11)}

        for h in range(10):
            if h == 0:
                ideal = 12 / lns[h]
            else:
                ideal = ideal + 1 / lns[h]

        denominador = np.ones_like(numerador)       
        denominador = denominador * np.array(list(lns.values()))

        DCG = numerador / denominador
        DCG = DCG.sum(axis=1)

        iDCG = np.ones_like(DCG) * ideal
        nDCG = DCG / iDCG
        nDGC_version.append({'estructura': v, 'performance': nDCG.mean(), 'performance_puntual': nDCG})
        print(v)
        print(nDCG.mean())
                
        #Selección.
        if max_nDCG:
            if nDCG.mean() > max_nDCG:
                max_nDCG = nDCG.mean() 
                s_feature = c_feature
        else:
            max_nDCG = nDCG.mean()
            s_feature = c_feature
            
    grupoA.remove(s_feature)
    grupoB.append(s_feature)
    
    versiones += c_versiones
    c_versiones = []
    for i in grupoA:
        c_versiones.append(grupoB + [i])


['dim_miv', 'dim_tbc', 'dim_liv', 'dim_sss', 'dim_vc', 'dim_ocat_16', 'dim_tbp', 'dim_bc']
0.2893094811953384



In [15]:
VERSIONES = pd.DataFrame(versiones)
VERSIONES.to_csv('versiones_29-11-20_v2.csv', index=False, sep=';')

In [14]:
#Anterior: 0.28953421814349656. Último: 0.2895673580691889.
seleccion_modelos = pd.DataFrame(np.array([x['performance_puntual'] for x in nDGC_version]).transpose())
seleccion_modelos = pd.concat([seleccion_modelos, pd.DataFrame(cluster_test, columns=['cluster'])], axis=1)
a = seleccion_modelos.groupby(['cluster']).mean().max(axis=1)
b = seleccion_modelos.groupby(['cluster']).count().max(axis=1)
c = seleccion_modelos.groupby(['cluster']).count().max(axis=1).sum()
sum((a * b) / c)

0.2895673580691889

In [16]:
send = seleccion_modelos.groupby(['cluster']).mean()
send.to_csv('seleccion_modelos_29-11-20_v2.csv', index=False, sep=';')

In [22]:
seleccion_modelos = pd.DataFrame(np.array([x['performance_puntual'] for x in nDGC_version]).transpose())
seleccion_modelos = pd.concat([seleccion_modelos, pd.DataFrame(cluster_test1, columns=['cluster'])], axis=1)
a = seleccion_modelos.groupby(['cluster']).mean().max(axis=1)
b = seleccion_modelos.groupby(['cluster']).count().max(axis=1)
c = seleccion_modelos.groupby(['cluster']).count().max(axis=1).sum()
sum((a * b) / c)

0.27575719113402736

In [20]:
send

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
cluster,,,,,,,,,,,,,,,,,,,,,
0,0.180681,0.179144,0.175962,0.180915,0.179230,0.176360,0.181129,0.179322,0.176048,0.180868,...,0.181307,0.178629,0.182209,0.180736,0.178933,0.181717,0.181104,0.181592,0.183598,0.182152
1,0.036731,0.003379,0.003379,0.003379,0.003379,0.003379,0.003379,0.003379,0.003379,0.003379,...,0.051158,0.051158,0.051158,0.051158,0.051158,0.051158,0.051158,0.051158,0.051158,0.051158
2,0.234303,0.236685,0.229413,0.240406,0.236478,0.229525,0.240757,0.236464,0.229455,0.239750,...,0.240914,0.238081,0.242812,0.239858,0.238052,0.241498,0.238729,0.239422,0.240870,0.242207
3,0.223538,0.219373,0.217434,0.221392,0.219709,0.218144,0.222369,0.219819,0.217139,0.221679,...,0.224699,0.222912,0.225466,0.225459,0.223828,0.226380,0.224836,0.224684,0.227236,0.225665
4,0.309294,0.310614,0.304315,0.313609,0.310420,0.304573,0.314263,0.310081,0.303378,0.313007,...,0.313917,0.313799,0.315113,0.311354,0.312996,0.311378,0.312969,0.313468,0.313102,0.314885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0.341468,0.393540,0.398557,0.395756,0.341073,0.398557,0.395756,0.398557,0.398557,0.398557,...,0.398672,0.398672,0.398672,0.398672,0.398672,0.398672,0.398672,0.398672,0.398672,0.398672
249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
252,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
modelos = np.array([x['performance_puntual'] for x in nDGC_version]).transpose()
inventario = {}
for c in np.unique(cluster_test):
    midx = modelos[cluster_test == c, :].mean(axis=0).argmax()
    inventario.update({c: {'v_id': midx, 'v_desc': versiones[midx]}})

In [17]:
#Predicción post.
k = 10

y_test = []
for i in tqdm(range(ib_test.size)):
    ccluster = cluster_test[i]
    #ccluster1 = cluster_test1[i]
    cdom = dom_test[i]
    cmket = mket_test[i]
    if ccluster in inventario:
        v = inventario[ccluster]['v_desc']
        #v = ['dim_bc', 'dim_liv', 'dim_miv', 'dim_ocat_16', 'dim_tbc', 'dim_tbc2', 'dim_tbp']
    else:
        alt_cluster = [x for x in list(inventario) if abs(x - ccluster) == min([abs(x - ccluster) for x in list(inventario)])][0]
        v = inventario[alt_cluster]['v_desc']
        #v = ['dim_bc', 'dim_liv', 'dim_miv', 'dim_ocat_16', 'dim_tbc', 'dim_tbc2', 'dim_tbp']
    for j, f in enumerate(v):
        if ccluster in grilla:
            cref = grilla[ccluster][f]
        else:
            alt_cluster = [x for x in grilla if abs(x - ccluster) == min([abs(x - ccluster) for x in grilla])][0]
            cref = grilla[alt_cluster][f]
        citems = pred_test[f][i]
        mask = (cref > 0) & (citems >= 0)
        cref = cref[mask]
        citems = citems[mask]
        if j == 0:
            ranks = cref
            items = citems
        else:
            ranks = np.concatenate([ranks, cref])
            items = np.concatenate([items, citems])
    y_hat = list(get_y_hat(items, ranks)[::-1])
    if len(y_hat) < k:
        size = k - len(y_hat)
        ext = list(gen_sales(y_hat, cdom, cmket)[:size])
        y_hat.extend(ext)
        if len(y_hat) < k:
            size = k - len(y_hat)
            if cmket == 1:
                ext = [x for x in top_mex if x not in y_hat][:size]
            else:
                ext = [x for x in top_brs if x not in y_hat][:size]
            y_hat.extend(ext)
    y_test.append(y_hat)

Y_TEST = pd.DataFrame(y_test, columns=['pred'+str(i) for i in range(k)])
Y_TEST_DO, y_test_do, Y_TEST_MA, y_test_ma = get_doms_and_mket(Y_TEST)
cond1 = compare(np.array(y_test), ib_test.reshape(ib_test.size,1).repeat(10, axis=1))
cond2 = compare(y_test_do, ib_do_test.reshape(ib_do_test.size,1).repeat(10, axis=1))
cond3 = y_test_ma == ib_ma_test.reshape(ib_ma_test.size,1).repeat(10, axis=1)
numerador = cond1 * 12 + cond2 * cond3
numerador[numerador > 12] = 12

lns = {0: np.log(2),
        1: np.log(3),
        2: np.log(4),
        3: np.log(5),
        4: np.log(6),
        5: np.log(7),
        6: np.log(8),
        7: np.log(9),
        8: np.log(10),
        9: np.log(11)}

for h in range(10):
    if h == 0:
        ideal = 12 / lns[h]
    else:
        ideal = ideal + 1 / lns[h]

denominador = np.ones_like(numerador)       
denominador = denominador * np.array(list(lns.values()))

DCG = numerador / denominador
DCG = DCG.sum(axis=1)

iDCG = np.ones_like(DCG) * ideal
nDCG = DCG / iDCG
nDGC_version.append({'estructura': v, 'performance': nDCG.mean(), 'performance_puntual': nDCG})
print(v)
print(nDCG.mean())


['dim_miv', 'dim_tbc', 'dim_liv', 'dim_sss', 'dim_vc', 'dim_ocat_16', 'dim_tbp', 'dim_bc', 'dim_opro_16']
0.2895342181434993


In [23]:
NDCG = pd.DataFrame(nDCG, columns=(['last_ndcg']))
NDCG.to_csv('last_ndcg8.csv', sep=';', index=False)

In [24]:
Y_TEST.to_csv('y_test8.csv', sep=';', index=False)

In [25]:
NUMERADOR = pd.DataFrame(numerador, columns=['puntaje'+str(i) for i in range(10)])
NUMERADOR.to_csv('last_numerador8.csv', sep=';', index=False)

In [26]:
NDGC = pd.DataFrame(nDGC_version)
NDGC.to_csv('ndgc_29-11-20_v1.csv', index=False, sep=';')

In [18]:
#Mercado visitado.
DFF_PRED = pd.read_csv('dff_pred.csv', sep=';')
mket_pred = np.array(DFF_PRED['mercado'])
DMV_PRED = pd.read_csv('dmv_pred.csv', sep=';')
dom_pred = np.array(DMV_PRED['vdomain']).reshape(-1)

In [19]:
#Clusters.
clusters = pd.read_csv('clusters_reloaded.csv', sep=';')
train_test = pd.read_csv('meli_segment_v2.csv', sep=';')
clusters = pd.concat([clusters['cs32'], train_test], axis=1)
clusters = clusters.loc[clusters['segment'] == 'test']
cluster_pred = np.array(clusters['cs32']).astype('int32')
cl_pred = np.unique(cluster_pred, return_counts=True)

In [32]:
#Trampa cluster.
clusters = pd.read_csv('dfs_pred.csv', sep=';')
clusters['segmento'] = 0
clusters.loc[clusters['v_len'] != 0, 'segmento'] = 1
cluster_pred1 = np.array(clusters['segmento']).astype('int32')
cl_pred1 = np.unique(cluster_pred1, return_counts=True)

In [20]:
#chosen_predictors = ['dim_bc',
#                     'dim_liv',
#                     'dim_miv',
#                     'dim_opro_13',
#                     'dim_ocat_16',
#                     'dim_scat_31',
#                     'dim_spro_13',
#                     'dim_sdom_31',
#                     'dim_tbd',
#                     'dim_tbd2',
#                     'dim_tbc',
#                     'dim_tbc2',
#                     'dim_tbp',
#                     'dim_tbp2',
#                     'dim_vc'] 

PRED_PRED, pred_pred = load_chosen_predictors_pred('C:/data/meli_challenge/predictors_pred', chosen_predictors)

In [21]:
#Predicción final optimizada.
k = 10
    
y_pred = []
for i in tqdm(range(mket_pred.size)):
    ccluster = cluster_pred[i]
    cdom = dom_pred[i]
    cmket = mket_pred[i]
    if ccluster in list(inventario):
        v = inventario[ccluster]['v_desc']
    else:
        alt_cluster = [x for x in list(inventario) if abs(x - ccluster) == min([abs(x - ccluster) for x in list(inventario)])][0]
        v = inventario[alt_cluster]['v_desc']
    for j, f in enumerate(v):
        cref = grilla[ccluster][f]
        citems = pred_pred[f][i]
        mask = (cref > 0) & (citems >= 0)
        cref = cref[mask]
        citems = citems[mask]
        if j == 0:
            ranks = cref
            items = citems
        else:
            ranks = np.concatenate([ranks, cref])
            items = np.concatenate([items, citems])
    y_hat = list(get_y_hat(items, ranks)[::-1])
    if len(y_hat) < k:
        size = k - len(y_hat)
        ext = list(gen_sales(y_hat, cdom, cmket)[:size])
        y_hat.extend(ext)
        if len(y_hat) < k:
            size = k - len(y_hat)
            if cmket == 1:
                ext = [x for x in top_mex if x not in y_hat][:size]
            else:
                ext = [x for x in top_brs if x not in y_hat][:size]
            y_hat.extend(ext)
    y_pred.append(y_hat)

In [22]:
#Export.
Y_PRED=pd.DataFrame(y_pred)
Y_PRED.to_csv('submission_n25.csv', index=False, header=False)